In this exercise we will use Spark to explore the hypothesis stated below:

**The number of farmer's markets in a given zip code can be predicted from the income and taxes paid in a given area.**

We explore the process for discovering whether or not we can accurately predict the number of farmer's markets.  We will use two datasets.

![img](http://training.databricks.com/databricks_guide/USDA_logo.png)

The [**Farmers Markets Directory and Geographic Data**](http://catalog.data.gov/dataset/farmers-markets-geographic-data/resource/cca1cc8a-9670-4a27-a8c7-0c0180459bef) dataset contains information on the longitude and latitude, state, address, name, and zip code of Farmers Markets in the United States. The raw data is published by the Department of Agriculture. 

![img](http://training.databricks.com/databricks_guide/irs-logo.jpg)

The [**SOI Tax Stats - Individual Income Tax Statistics - ZIP Code Data (SOI)**](http://catalog.data.gov/dataset/zip-code-data) study provides detailed tabulations of individual income tax return data at the state and ZIP code level and is provided by the IRS. The data includes items, such as:

- Number of returns, which approximates the number of households
- Number of personal exemptions, which approximates the population
- Adjusted gross income
- Wages and salaries
- Dividends before exclusion
- Interest received

Read in the data

This data is located in in csv files and Apache Spark 2.0 can read the data in directly.

In [3]:
taxes2013 = spark.read\
  .option("header", "true")\
  .csv("dbfs:/databricks-datasets/data.gov/irs_zip_code_data/data-001/2013_soi_zipcode_agi.csv")

In [4]:
markets = spark.read\
  .option("header", "true")\
  .csv("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv")

Use the `display` command to take a quick look at the dataframes you created.

In [6]:
display(taxes2013)

STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
01,AL,00000,1,870380.0000,488030.0000,122290.0000,247000.0000,500770.0000,1452580.0000,571240.0000,11255896.000,870380.0000,11444868.000,700700.0000,8889326.0000,103290.0000,77952.0000,46870.0000,75071.0000,40890.0000,47416.0000,15650.0000,6538.0000,146240.0000,824487.0000,37970.0000,23583.0000,38400.0000,221790.0000,111060.0000,1066291.0000,8800.0000,49720.0000,187559.0000,35370.0000,62791.0000,8980.0000,10323.0000,155190.0000,186574.0000,3300.0000,747.0000,140.0000,487.0000,8950.0000,33584.0000,2540.0000,6212.0000,20760.0000,17533.0000,5900.0000,16956.0000,30.0000,2.0000,57090.0000,794815.0000,884758.0000,23440.0000,22991.0000,25940.0000,20686.0000,29140.0000,25144.0000,54970.0000,84317.0000,27140.0000,151005.0000,42500.0000,111909.0000,338560.0000,1874627.0000,333040.0000,196535.0000,0.0000,0.0000,108740.0000,37220.0000,2400.0000,56.0000,8850.0000,2899.0000,33010.0000,16969.0000,32080.0000,5457.0000,33530.0000,10160.0000,2650.0000,677.0000,117420.0000,152943.0000,807980.0000,2277816.0000,399710.0000,1174659.0000,371280.0000,1057938.0000,258200.0000,327115.0000,80350.0000,76412.0000,255750.0000,159189.0000,371450.0000,318777.0000,0.0000,0.0000,0.0000,0.0000,59580.0000,44367.0000,767170.0000,2005593.0000
01,AL,00000,2,490960.0000,195840.0000,155230.0000,125280.0000,286130.0000,1027850.0000,383240.0000,17632481.000,490960.0000,17810952.000,427730.0000,14501798.000,97400.0000,81216.0000,42440.0000,92536.0000,37100.0000,61479.0000,58830.0000,29891.0000,61400.0000,252768.0000,32710.0000,54639.0000,34860.0000,320115.0000,101580.0000,1764445.0000,8890.0000,23590.0000,84183.0000,87130.0000,542763.0000,10690.0000,49672.0000,95190.0000,177106.0000,12310.0000,2968.0000,240.0000,975.0000,6870.0000,31725.0000,6350.0000,20932.0000,36070.0000,36540.0000,2360.0000,5915.0000,60.0000,35.0000,115520.0000,1717953.0000,4390768.0000,83910.0000,140609.0000,25240.0000,32077.0000,77400.0000,61241.0000,113360.0000,262971.0000,72830.0000,405101.0000,94310.0000,341396.0000,470370.0000,8602452.0000,468160.0000,1038975.0000,0.0000,0.0000,217760.0000,194669.0000,6030.0000,262.0000,28850.0000,16248.0000,46050.0000,47721.0000,65150.0000,12104.0000,124900.0000,110801.0000,12820.0000,4944.0000,37210.0000,77948.0000,480870.0000,2041535.0000,132780.0000,259313.0000,112050.0000,223466.0000,104610.0000,156274.0000,37270.0000,34254.0000,367070.0000,844185.0000,386570.0000,935430.0000,0.0000,0.0000,0.0000,0.0000,65850.0000,94281.0000,418070.0000,1192755.0000
01,AL,00000,3,258810.0000,72710.0000,146880.0000,32860.0000,157670.0000,594910.0000,189340.0000,15916085.000,258810.0000,16070153.000,223860.0000,12289284.000,86850.0000,80627.0000,40800.0000,118256.0000,36250.0000,81316.0000,68480.0000,42607.0000,39840.0000,259836.0000,32420.0000,84137.0000,27970.0000,348231.0000,72490.0000,1727323.0000,7940.0000,11640.0000,45125.0000,59580.0000,849865.0000,10940.0000,94229.0000,68820.0000,151722.0000,9550.0000,2369.0000,190.0000,1100.0000,4800.0000,24606.0000,5070.0000,20023.0000,26750.0000,26173.0000,2290.0000,5165.0000,110.0000,171.0000,105460.0000,1797725.0000,6537061.0000,85350.0000,216962.0000,16720.0000,25271.0000,85670.0000,78046.0000,104930.0000,347945.0000,79910.0000,498801.0000,90350.0000,3961

Now register the DataFrames as Spark SQL tables so that we can use our SQL skills to manipulate the data. The lifetime of this temporary table is tied to the Spark Context that was used to create this DataFrame. When you shutdown the SQLContext associated with a cluster the temporary table disappears as well.

In [8]:
taxes2013.createOrReplaceTempView("taxes2013")
markets.createOrReplaceTempView("markets")

## Running SQL Commands

This is a python notebook.  To use another language in a python notebook we prefix the cell with the language identifier.  To use SQL in your python notebook, prefix the cell with `%sql`

Use SQL to `show tables`.

In [10]:
%sql show tables

database,tableName,isTemporary
default,test_psv,false
,markets,true
,taxes2013,true


Take a quick look at the tables using `SELECT *`.  This is very similar to calling `display` on the DataFrame.

In [12]:
%sql SELECT * FROM taxes2013

STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
01,AL,00000,1,870380.0000,488030.0000,122290.0000,247000.0000,500770.0000,1452580.0000,571240.0000,11255896.000,870380.0000,11444868.000,700700.0000,8889326.0000,103290.0000,77952.0000,46870.0000,75071.0000,40890.0000,47416.0000,15650.0000,6538.0000,146240.0000,824487.0000,37970.0000,23583.0000,38400.0000,221790.0000,111060.0000,1066291.0000,8800.0000,49720.0000,187559.0000,35370.0000,62791.0000,8980.0000,10323.0000,155190.0000,186574.0000,3300.0000,747.0000,140.0000,487.0000,8950.0000,33584.0000,2540.0000,6212.0000,20760.0000,17533.0000,5900.0000,16956.0000,30.0000,2.0000,57090.0000,794815.0000,884758.0000,23440.0000,22991.0000,25940.0000,20686.0000,29140.0000,25144.0000,54970.0000,84317.0000,27140.0000,151005.0000,42500.0000,111909.0000,338560.0000,1874627.0000,333040.0000,196535.0000,0.0000,0.0000,108740.0000,37220.0000,2400.0000,56.0000,8850.0000,2899.0000,33010.0000,16969.0000,32080.0000,5457.0000,33530.0000,10160.0000,2650.0000,677.0000,117420.0000,152943.0000,807980.0000,2277816.0000,399710.0000,1174659.0000,371280.0000,1057938.0000,258200.0000,327115.0000,80350.0000,76412.0000,255750.0000,159189.0000,371450.0000,318777.0000,0.0000,0.0000,0.0000,0.0000,59580.0000,44367.0000,767170.0000,2005593.0000
01,AL,00000,2,490960.0000,195840.0000,155230.0000,125280.0000,286130.0000,1027850.0000,383240.0000,17632481.000,490960.0000,17810952.000,427730.0000,14501798.000,97400.0000,81216.0000,42440.0000,92536.0000,37100.0000,61479.0000,58830.0000,29891.0000,61400.0000,252768.0000,32710.0000,54639.0000,34860.0000,320115.0000,101580.0000,1764445.0000,8890.0000,23590.0000,84183.0000,87130.0000,542763.0000,10690.0000,49672.0000,95190.0000,177106.0000,12310.0000,2968.0000,240.0000,975.0000,6870.0000,31725.0000,6350.0000,20932.0000,36070.0000,36540.0000,2360.0000,5915.0000,60.0000,35.0000,115520.0000,1717953.0000,4390768.0000,83910.0000,140609.0000,25240.0000,32077.0000,77400.0000,61241.0000,113360.0000,262971.0000,72830.0000,405101.0000,94310.0000,341396.0000,470370.0000,8602452.0000,468160.0000,1038975.0000,0.0000,0.0000,217760.0000,194669.0000,6030.0000,262.0000,28850.0000,16248.0000,46050.0000,47721.0000,65150.0000,12104.0000,124900.0000,110801.0000,12820.0000,4944.0000,37210.0000,77948.0000,480870.0000,2041535.0000,132780.0000,259313.0000,112050.0000,223466.0000,104610.0000,156274.0000,37270.0000,34254.0000,367070.0000,844185.0000,386570.0000,935430.0000,0.0000,0.0000,0.0000,0.0000,65850.0000,94281.0000,418070.0000,1192755.0000
01,AL,00000,3,258810.0000,72710.0000,146880.0000,32860.0000,157670.0000,594910.0000,189340.0000,15916085.000,258810.0000,16070153.000,223860.0000,12289284.000,86850.0000,80627.0000,40800.0000,118256.0000,36250.0000,81316.0000,68480.0000,42607.0000,39840.0000,259836.0000,32420.0000,84137.0000,27970.0000,348231.0000,72490.0000,1727323.0000,7940.0000,11640.0000,45125.0000,59580.0000,849865.0000,10940.0000,94229.0000,68820.0000,151722.0000,9550.0000,2369.0000,190.0000,1100.0000,4800.0000,24606.0000,5070.0000,20023.0000,26750.0000,26173.0000,2290.0000,5165.0000,110.0000,171.0000,105460.0000,1797725.0000,6537061.0000,85350.0000,216962.0000,16720.0000,25271.0000,85670.0000,78046.0000,104930.0000,347945.0000,79910.0000,498801.0000,90350.0000,3961

Next, cleanup the data using SQL
1. Create a new table called `cleaned_taxes` from the `taxes2013` temp table
1. All the values are currently strings, convert them to approriate data types as needed
  1. zipcode => int
  1. mars1 => int
  1. mars2 => int
  1. numdep => int
  1. A02650 => double
  1. A00300 => double
  1. a01000 => double
  1. a00900 => double 
1. Rename columns for clarity as needed
  1. state => state
  1. zipcode => zipcode
  1. mars1 => single_returns
  1. mars2 => joint_returns
  1. numdep => numdep
  1. A02650 => total_income_amount
  1. A00300 => taxable_interest_amount
  1. a01000 => net_capital_gains
  1. a00900 => biz_net_income
1. Shorten each zip code to 4 digits instead of 5, to group nearby areas together

In [14]:
%sql
DROP TABLE IF EXISTS cleaned_taxes;

CREATE TABLE cleaned_taxes AS
SELECT 
  state, 
  int(zipcode / 10) as zipcode,
  int(mars1) as single_returns,
  int(mars2) as joint_returns,
  int(numdep) as numdep,
  double(A02650) as total_income_amount,
  double(A00300) as taxable_interest_amount,
  double(a01000) as net_capital_gains,
  double(a00900) as biz_net_income
FROM taxes2013

Now that the data is cleaned up, create some nice plots

Explore the average total income per zip code per state. 

Which states have the highest income per zip code?

In [16]:
%sql SELECT state, total_income_amount FROM cleaned_taxes 

state,total_income_amount
AL,1.1444868E7
AL,1.7810952E7
AL,1.6070153E7
AL,1.4288572E7
AL,2.605392E7
AL,2.0752068E7
AL,19851.0
AL,49338.0
AL,56170.0
AL,52977.0


New Jersey and California have higher average incomes per zip code.

Next let's explore some specifics of this particular dataset.

Use SQL to `describe` the dataset so that you can see the schema.

In [19]:
%sql describe cleaned_taxes

col_name,data_type,comment
state,string,null
zipcode,int,null
single_returns,int,null
joint_returns,int,null
numdep,int,null
total_income_amount,double,null
taxable_interest_amount,double,null
net_capital_gains,double,null
biz_net_income,double,null


Let's look at the set of zip codes with the lowest total capital gains and plot the results.

In [21]:
%sql
SELECT zipcode, SUM(net_capital_gains) AS cap_gains
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY cap_gains ASC
LIMIT 10

zipcode,cap_gains
2659,-20.0
4126,-14.0
8133,-13.0
1271,-7.0
8742,-4.0
4186,0.0
4161,0.0
4092,0.0
2656,0.0
2622,0.0


Let's look at a combination of capital gains and business net income to see what we find. 

Build a `combo` metric that represents the total capital gains and business net income by zip code.

In [23]:
%sql
SELECT zipcode,
  SUM(biz_net_income) as business_net_income,
  SUM(net_capital_gains) as capital_gains,
  SUM(net_capital_gains) + SUM(biz_net_income) as capital_and_business_income
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY capital_and_business_income DESC
LIMIT 50

zipcode,business_net_income,capital_gains,capital_and_business_income
1002,1866926.0,1.2443093E7,1.4310019E7
1001,1491999.0,8606053.0,1.0098052E7
9430,349156.0,5976580.0,6325736.0
9402,681523.0,4667696.0,5349219.0
9411,1027768.0,3929969.0,4957737.0
1000,685593.0,4110129.0,4795722.0
6061,793338.0,3851723.0,4645061.0
683,277152.0,3964177.0,4241329.0
1012,331281.0,2426170.0,2757451.0
7702,541319.0,2186449.0,2727768.0


We can also get a peek at what will happen when we execute the query.

Use the `EXPLAIN` keyword in SQL.

In [25]:
%sql
EXPLAIN
  SELECT zipcode,
    SUM(biz_net_income) as net_income,
    SUM(net_capital_gains) as cap_gains,
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

plan
"== Physical Plan == TakeOrderedAndProject(limit=50, orderBy=[combo#2033 DESC NULLS LAST], output=[zipcode#1967,net_income#2031,cap_gains#2032,combo#2033]) +- *(2) HashAggregate(keys=[zipcode#1967], functions=[finalmerge_sum(merge sum#2043) AS sum(biz_net_income#1974)#2037, finalmerge_sum(merge sum#2045) AS sum(net_capital_gains#1973)#2038]) +- Exchange hashpartitioning(zipcode#1967, 200) +- *(1) HashAggregate(keys=[zipcode#1967], functions=[partial_sum(biz_net_income#1974) AS sum#2043, partial_sum(net_capital_gains#1973) AS sum#2045]) +- *(1) Project [zipcode#1967, net_capital_gains#1973, biz_net_income#1974] +- *(1) Filter ((isnotnull(zipcode#1967) && NOT (zipcode#1967 = 0)) && NOT (zipcode#1967 = 9999)) +- *(1) FileScan parquet default.cleaned_taxes[zipcode#1967,net_capital_gains#1973,biz_net_income#1974] Batched: true, Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/cleaned_taxes], PartitionFilters: [], PushedFilters: [IsNotNull(zipcode), Not(EqualTo(zipcode,0)), Not(EqualTo(zipcode,9999))], ReadSchema: struct"


We can see above that we're fetching the data from `dbfs:/user/hive/warehouse/cleaned_taxes` which is where the data is stored when we registered it as a temporary table. 

Let's `cache` the data

In [27]:
%sql CACHE TABLE cleaned_taxes

When we cache data using SQL, Spark caches *eagerly*--right away.  This differs from `cacheTable` on the `SqlContext` which caches the data only when it is needed.

Let's run the exact same query again. You'll notice that it takes just a fraction of the time because the data is stored in memory.

In [29]:
%sql
SELECT zipcode,
  SUM(biz_net_income) as net_income,
  SUM(net_capital_gains) as cap_gains,
  SUM(net_capital_gains) + SUM(biz_net_income) as combo
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY combo desc
limit 50

zipcode,net_income,cap_gains,combo
1002,1866926.0,1.2443093E7,1.4310019E7
1001,1491999.0,8606053.0,1.0098052E7
9430,349156.0,5976580.0,6325736.0
9402,681523.0,4667696.0,5349219.0
9411,1027768.0,3929969.0,4957737.0
1000,685593.0,4110129.0,4795722.0
6061,793338.0,3851723.0,4645061.0
683,277152.0,3964177.0,4241329.0
1012,331281.0,2426170.0,2757451.0
7702,541319.0,2186449.0,2727768.0


Now `EXPLAIN` the plan

In [31]:
%sql
EXPLAIN
  SELECT zipcode,
    SUM(biz_net_income) as net_income,
    SUM(net_capital_gains) as cap_gains,
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

plan
"== Physical Plan == TakeOrderedAndProject(limit=50, orderBy=[combo#2248 DESC NULLS LAST], output=[zipcode#1967,net_income#2246,cap_gains#2247,combo#2248]) +- *(2) HashAggregate(keys=[zipcode#1967], functions=[finalmerge_sum(merge sum#2303) AS sum(biz_net_income#1974)#2252, finalmerge_sum(merge sum#2305) AS sum(net_capital_gains#1973)#2253]) +- Exchange hashpartitioning(zipcode#1967, 200) +- *(1) HashAggregate(keys=[zipcode#1967], functions=[partial_sum(biz_net_income#1974) AS sum#2303, partial_sum(net_capital_gains#1973) AS sum#2305]) +- *(1) Filter ((isnotnull(zipcode#1967) && NOT (zipcode#1967 = 0)) && NOT (zipcode#1967 = 9999)) +- InMemoryTableScan [zipcode#1967, net_capital_gains#1973, biz_net_income#1974], [isnotnull(zipcode#1967), NOT (zipcode#1967 = 0), NOT (zipcode#1967 = 9999)] +- InMemoryRelation [state#1966, zipcode#1967, single_returns#1968, joint_returns#1969, numdep#1970, total_income_amount#1971, taxable_interest_amount#1972, net_capital_gains#1973, biz_net_income#1974], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas), `cleaned_taxes` +- *(1) FileScan parquet default.cleaned_taxes[state#1966,zipcode#1967,single_returns#1968,joint_returns#1969,numdep#1970,total_income_amount#1971,taxable_interest_amount#1972,net_capital_gains#1973,biz_net_income#1974] Batched: true, Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/cleaned_taxes], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<state:string,zipcode:int,single_returns:int,joint_returns:int,numdep:int,total_income_amou..."


Instead of going down to the source data it performs a `InMemoryColumnarTableScan`. This means that it has all of the information that it needs in memory.

Now let's look at the Farmer's Market Data. 

Start with a total summation of farmer's markets per state.

In [33]:
%sql SELECT State, COUNT(State) as Sum
      FROM markets 
      GROUP BY State 

State,Sum
Utah,41
Hawaii,97
Minnesota,189
Ohio,321
Oregon,171
Arkansas,100
Texas,205
North Dakota,65
null,0
Pennsylvania,303
